# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# load csv to dataframe
city_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
city_data.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,mount gambier,-37.83,140.77,44.60,87,90,2.10,AU
1,butaritari,3.07,172.79,82.76,77,1,6.22,KI
2,tiznit,29.58,-9.50,96.49,22,0,3.28,MA
3,aquidauana,-20.47,-55.79,86.13,29,90,0.99,BR
4,wenatchee,47.42,-120.31,73.40,49,1,6.70,US


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gmaps
gmaps.configure(api_key=g_key)

# Lat and Lng as locations and humidity as weight
locations = city_data[["Lat", "Lng"]]
humidity = city_data["Humidity"]

In [4]:
# add heatmap layer to map and display 
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, 
                                 max_intensity=100, point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# narrow cities to ideal weather conditions
index = city_data[(city_data["Humidity"] > 45) | \
                  (city_data["Wind Speed"] > 5) | \
                  (city_data["Max Temp"] < 70) | \
                  (city_data["Max Temp"] > 80) | \
                  (city_data["Cloudiness"] > 20)].index.tolist()
hotel_df = city_data.drop(index)

# drop null values and change City to title case
hotel_df = hotel_df.dropna()
hotel_df["City"] = hotel_df["City"].str.title()
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
6,Pampierstad,-27.78,24.69,71.24,17,0,0.68,ZA
14,Bobrovytsya,50.75,31.38,73.40,35,15,5.00,UA
152,Mount Darwin,-16.77,31.58,72.64,30,0,2.59,ZW
300,Dekar,-21.53,21.93,72.01,20,0,3.18,BW
435,Evreux,49.02,1.15,78.01,29,0,4.10,FR
472,Bestobe,52.50,73.10,73.72,42,0,3.43,KZ
480,Usinsk,65.99,57.53,79.79,40,0,3.02,RU
485,Sovetskiy,61.36,63.58,79.70,34,0,1.21,RU
553,Lubango,-14.92,13.49,71.60,18,0,2.10,AO


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# add column to dataframe
hotel_df["Hotel Name"] = ""

In [7]:
# set parameters to search for hotels within 5 km
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "keyword": "Hotel",
    "radius": 5000,
    "type": "hotel",
    "key": g_key
}

# use nearby search to find first hotel near city and store into dataframe
for index, row in hotel_df.iterrows():
    params["location"] = f"{row['Lat']},{row['Lng']}"
    response = requests.get(base_url, params=params).json()
    results = response["results"]
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    except (KeyError, IndexError):
        print(f"There is no hotel within 5 km of {row['City']}.")

There is no hotel within 5 km of Pampierstad.
There is no hotel within 5 km of Mount Darwin.
There is no hotel within 5 km of Dekar.
There is no hotel within 5 km of Bestobe.


In [8]:
# display results
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
6,Pampierstad,-27.78,24.69,71.24,17,0,0.68,ZA,
14,Bobrovytsya,50.75,31.38,73.40,35,15,5.00,UA,"Готель ""Трубіж"""
152,Mount Darwin,-16.77,31.58,72.64,30,0,2.59,ZW,
300,Dekar,-21.53,21.93,72.01,20,0,3.18,BW,
435,Evreux,49.02,1.15,78.01,29,0,4.10,FR,Normandy Hotel
472,Bestobe,52.50,73.10,73.72,42,0,3.43,KZ,
480,Usinsk,65.99,57.53,79.79,40,0,3.02,RU,"Gostinitsa ""Uyutnyy Dom"""
485,Sovetskiy,61.36,63.58,79.70,34,0,1.21,RU,Usad'ba
553,Lubango,-14.92,13.49,71.60,18,0,2.10,AO,Hotel Lubango


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in \
              hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer on top of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))